In [1]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
import copy
import math

from PIL import Image
from matplotlib import image

import pickle
import time

import pybullet as p
from envs import FrankaEnv
from envs.robot_env import RobotEnv
from envs import robot_sim
from utils.bullet_utils import add_goal, add_obstacle_ball
from utils.python_utils import merge_dicts

#import controller_track_taskspace as contr
from pybullet_helpers import *
from trajectory_generator import *
from extract_density import *
import pytorch_kinematics as pk
device='cpu'

pybullet build time: May 20 2022 19:44:17


In [2]:
cl_env=CleanFrankaEnv(mobile=True)
cl_env.set_up_bul(True)

True
mobile_franka
robot name: mobile_franka


In [3]:
config = {
    "q_init": [-1.1,-1.1, 0.03755247, 0.398266408, 
               0.38741508, 0.209020819,  0.922805379,
        0.46475231, 1.20273773,0.5],
}
con_q=torch.tensor(config["q_init"])
cur_joint_position=con_q
ind=0
for i in range(0,len(cur_joint_position)):
    _, joint_name, joint_type, _, _, _, _, _, _, _, _, _, _, _, _, _, _ = p.getJointInfo(cl_env._robot.robot_uid, ind)
    if(joint_type==4):
        ind=ind+1
    cl_env._p.resetJointState(cl_env._robot.robot_uid,ind,cur_joint_position[i])
    ind=ind+1

In [4]:
# test_len=np.array([0.1,2,1.8])/2
# c_pos=np.array([1,0.,0.75])
# c_ori=np.array([0,0,0,1.])
# deg_ang=39.6

In [5]:
with open('test_dat_dict.pkl', 'rb') as f:
    training_dat = pickle.load(f)
    dist_list_np=torch.tensor(training_dat['features'])
    activated_list=training_dat['prims']
    paths_list=training_dat['paths']
    

In [6]:
cur_env=env_iter_cur=130

id_list=[]
y_vals_cur=activated_list[cur_env]
for i in range(len(y_vals_cur)):
    if(y_vals_cur[i].type=='cylinder'):
        radius=y_vals_cur[i].r
        height=y_vals_cur[i].h
        c_pos=y_vals_cur[i].pos
        c_ori=y_vals_cur[i].orientation
        color=[0.0, 0.0, 1.0, 1]
        visual = p.createVisualShape(
            p.GEOM_CYLINDER,radius=radius,
            length=height,
            rgbaColor=color)
        obj=p.createMultiBody(baseMass=0,
                                    baseCollisionShapeIndex=-1,
                                    baseVisualShapeIndex=visual,
                                    basePosition=c_pos,
                                     baseOrientation=c_ori)
        id_list.append(obj)
    if(y_vals_cur[i].type=='cube'):
        test_len=y_vals_cur[i].lengths/2
        c_pos=y_vals_cur[i].pos
        c_ori=y_vals_cur[i].orientation
        color=[np.random.rand(1),  np.random.rand(1), np.random.rand(1), 1]
        visual = p.createVisualShape(
            p.GEOM_BOX, halfExtents=test_len, rgbaColor=color)
        obj=p.createMultiBody(baseMass=0,
                                    baseCollisionShapeIndex=-1,
                                    baseVisualShapeIndex=visual,
                                    basePosition=c_pos,
                                     baseOrientation=c_ori)
        id_list.append(obj)

In [7]:
# color=[np.random.rand(1),  np.random.rand(1), np.random.rand(1), 1]
# visual = p.createVisualShape(
#     p.GEOM_BOX, halfExtents=test_len, rgbaColor=color)
# obj=p.createMultiBody(baseMass=0,
#                                     baseCollisionShapeIndex=-1,
#                                     baseVisualShapeIndex=visual,
#                                     basePosition=c_pos,
#                                      baseOrientation=c_ori)

In [8]:
deg_ang=39.6

In [10]:
viewMatrix = p.computeViewMatrix(
    cameraEyePosition=[-1.8, 2., 0.5],
    cameraTargetPosition=[-0.,-1.1,0.75],
    cameraUpVector=[0, 0.05, 0.85]
)
projectionMatrix = p.computeProjectionMatrixFOV(
    fov=deg_ang,
    aspect=1.0,
    nearVal=1.,
    farVal=6.)
width, height, rgbImg, depthImg, segImg = p.getCameraImage(
    width=600, 
    height=600,
    viewMatrix=viewMatrix,
    projectionMatrix=projectionMatrix)

In [11]:
img = Image.fromarray(rgbImg[:,:,:3], "RGB")
img.show()
img.save("mobile_view_0.png")

In [12]:
target_pose=torch.tensor(viewMatrix).reshape((4,4)).T
rot_mat=target_pose[:3,:3]
transl_vec=target_pose[:3,3]
rot_mat_inv=torch.inverse(rot_mat)
trans_move=-rot_mat_inv@transl_vec
pose_mat=torch.eye(4)
pose_mat[:3,:3]=rot_mat_inv
pose_mat[:3,3]=trans_move

In [13]:
viewMatrix = p.computeViewMatrix(
    cameraEyePosition=[-3., 1., 2.5],
    cameraTargetPosition=[-0.,-1.1,0.75],
    cameraUpVector=[0, 0.05, 0.85]
)
width, height, rgbImg, depthImg, segImg = p.getCameraImage(
    width=600, 
    height=600,
    viewMatrix=viewMatrix,
    projectionMatrix=projectionMatrix)

In [14]:
img = Image.fromarray(rgbImg[:,:,:3], "RGB")
img.show()
img.save("mobile_view_1.png")

In [15]:
target_pose=torch.tensor(viewMatrix).reshape((4,4)).T
rot_mat=target_pose[:3,:3]
transl_vec=target_pose[:3,3]
rot_mat_inv=torch.inverse(rot_mat)
trans_move=-rot_mat_inv@transl_vec
pose_mat2=torch.eye(4)
pose_mat2[:3,:3]=rot_mat_inv
pose_mat2[:3,3]=trans_move

In [16]:
poses=torch.vstack([pose_mat[None],pose_mat2[None]])
torch.save(poses,"poses_mobile.tar")